***Image pre-processing (True Color, False Color, NDVI)***

In [ ]:
# Creation of NDVI Image
import numpy as np
from PIL import Image

# Import images as numpy arrays
b04_img = Image.open('images/b04.jpg')
b08_img = Image.open('images/b08.jpg')

b04_array = np.array(b04_img)
b08_array = np.array(b08_img)

# Normalize the arrays to float32
b04_array = b04_array.astype(np.float32) / 255.0
b08_array = b08_array.astype(np.float32) / 255.0

# Calculate NDVI
ndvi = (b08_array - b04_array) / (b08_array + b04_array)

# Map NDVI values to 0-255 for visualization
ndvi_mapped = (((ndvi + 1) / 2)* 255).astype(np.uint8)

# Create RGB array (ndvi, ndvi, ndvi)
ndvi_rgb = np.stack((ndvi_mapped, ndvi_mapped, ndvi_mapped), axis=-1)

# Convert to PIL Image
ndvi_image = Image.fromarray(ndvi_rgb)

# Convert into 512x512
ndvi_image = ndvi_image.resize((512, 512), Image.LANCZOS)

# Save the NDVI image
ndvi_image.save('images/ndvi.jpg')


***Model import***

In [ ]:
from diffusers import ControlNetModel, StableDiffusionControlNetPipeline

controlnet = ControlNetModel.from_pretrained("mespinosami/sen12mscr-sd-1_5")
pipe = StableDiffusionControlNetPipeline.from_pretrained(
	"runwayml/stable-diffusion-v1-5", controlnet=controlnet
)

print(controlnet)


***Input images*** 
Pre processing of all the three different images and input in the model without touching the layers

In [11]:
true_color = Image.open('images/true_color.jpg')
false_color = Image.open('images/false_color.jpg')
ndvi_image = Image.open('images/ndvi.jpg')

# Convert into 512x512
true_color = true_color.resize((512, 512), Image.LANCZOS)
false_color = false_color.resize((512, 512), Image.LANCZOS)


# Print the sizes of the images
print(f"True Color Image Size: {true_color.size}")
print(f"False Color Image Size: {false_color.size}")
print(f"NDVI Image Size: {ndvi_image.size}")

# Print mode checks
print(f"True Color Image Mode: {true_color.mode}")
print(f"False Color Image Mode: {false_color.mode}")
print(f"NDVI Image Mode: {ndvi_image.mode}")

# Pipeline execution
prompt = 'a satellite view of some agricultural fields, in high-resolution, top-down view, near Rome'
controlnet_image = 'true_color'  # Options: 'true_color', 'false_color', 'ndvi'

output = pipe(
    prompt=prompt,
    image=ndvi_image,
    num_inference_steps=50
)

output_image = output.images[0]
# Save the output image
output_image.save(f'images/output_image' + '_ndvi_image' + '.jpg')


True Color Image Size: (512, 512)
False Color Image Size: (512, 512)
NDVI Image Size: (512, 512)
True Color Image Mode: RGB
False Color Image Mode: RGB
NDVI Image Mode: RGB


  0%|          | 0/50 [00:00<?, ?it/s]